In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras.datasets import mnist

In [20]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [21]:
print("train data (count, row, column) : " + str(X_train.shape) )
print("test data  (count, row, column) : " + str(X_test.shape) )
# 각각 가로 28, 세로 28픽셀로 구성된 손글씨 데이터가 있는거임

train data (count, row, column) : (60000, 28, 28)
test data  (count, row, column) : (10000, 28, 28)


In [22]:
print(X_train[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

In [23]:
"""
모델 학습 시작에 앞서, 데이터를 정규화합니다.
정규화는 입력값을 0부터 1의 값으로 변경하게 됩니다.
정규화된 입력값은 경사하강법으로 모델 학습 시, 보다 쉽고 빠르게 최적의 W,B를 찾는 데 도움을 줍니다.
"""
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [24]:
print(X_train[0])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.    

In [25]:
print("train target (count) : " + str(y_train.shape) )
print("test target  (count) : " + str(y_test.shape) )
y_train[0], y_test[0]

train target (count) : (60000,)
test target  (count) : (10000,)


(5, 7)

In [26]:
"""
이번 실습에서는 28*28 픽셀의 지역적인 정보를 사용하지 않고, 단순히 정규화된 입력값만을 가지고,
숫자 분류를 할 것이기 때문에, 행과 열의 구분 없이, 단순히 784 길이의 배열로 데이터를 단순화시킵니다.
"""
input_dim = 784 # 28 * 28
X_train = X_train.reshape(60000, input_dim)
X_test = X_test.reshape(10000, input_dim)

In [27]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
# 잘 바꼈네

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [28]:
"""
학습 시, y값과의 cross entropy를 측정해야하므로, 아래의 코드를 실행하여 y를 one hot encoding으로 변환시켜줍니다.
"""
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [29]:
print(y_train[0])
# 5였는데 one hot encoding으로 잘 바뀌었네

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [30]:
"""
케라스의 Sequential()을 사용하여 간단하게 소프트맥스를 구현할 수 있습니다.
총 784개(28*28)의 입력을 받아서, 10개의 시그모이드 값을 출력하는 모델을 아래의 코드를 실행하여 구현합니다.
"""
model = Sequential()
model.add(Dense(input_dim=input_dim, units=10, activation='softmax'))

In [31]:
"""
모델의 학습을 진행합니다.
10개의 클래스로 분류할 것이기 때문에, categorical_crossentropy를 비용함수로 사용한 경사하강법으로 최적의 W와 biases를 학습합니다.
"""
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=2048, epochs=100, verbose=False)

In [32]:
"""
테스트를 진행하여, 정확도를 측정합니다.
"""
score = model.evaluate(X_test, y_test) 
print('Test accuracy:', score[1])

10000/10000 [==============================] - 0s 15us/step
Test accuracy: 0.8895999789237976


In [33]:
"""
아래의 코드를 실행하여, 소프트맥스 모델의 구조를 쉽게 시각화 할 수 있습니다.
총 10개의 로지스틱회귀가 있고, 각 로지스틱회귀는 784개의 weight와 1개의 bias를 갖고 있기 때문에,
총 7850 (785*10)개의 Param이 있는 것을 보실 수 있습니다.
"""
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.layers[0].weights

[<tf.Variable 'dense_2/kernel:0' shape=(784, 10) dtype=float32, numpy=
 array([[-0.07109872, -0.03580489,  0.00235479, ..., -0.07376286,
          0.0665495 ,  0.03848638],
        [-0.02031428,  0.00983395, -0.01997942, ...,  0.00704253,
         -0.0457274 ,  0.04113939],
        [ 0.02687798,  0.05415878, -0.05050244, ..., -0.05671976,
         -0.0461891 , -0.02145062],
        ...,
        [-0.02955536,  0.08660119, -0.00690462, ..., -0.04464151,
         -0.03610282, -0.0495295 ],
        [ 0.04311577, -0.05007106,  0.07653914, ...,  0.02463417,
          0.08375672, -0.00936085],
        [-0.02850781,  0.05591425, -0.0261223 , ...,  0.07432287,
          0.06217998, -0.07200574]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(10,) dtype=float32, numpy=
 array([-0.07489419,  0.15636937, -0.04469553, -0.06176617,  0.04847243,
         0.16473685, -0.01398329,  0.11099518, -0.2550595 , -0.03017491],
       dtype=float32)>]